In [1]:
#Import GPT2Tokenizer to use

from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [2]:
import pickle
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from IPython.display import display
import sympy as sp
import urllib
sp.init_printing(use_latex=True)
import matplotlib.pyplot as plt
import textwrap
#import os
#os.environ['LD_LIBRARY_PATH']='/opt/conda/lib'

2023-06-19 07:12:37.852691: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
#Transormer block setup
#Initializes multi-head attention, feed-forward network, layer normalization
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1, **kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads,
                                                   key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="gelu"), #feed-forward
             keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


In [4]:
#Initialize transformer, multi-head-attention, ff net, and mask
class GPTransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1, **kwargs):
        super(GPTransformerBlock, self).__init__(**kwargs)
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads,
                                                   key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="gelu"),
             keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, x, training):
        y = x
        y = self.layernorm1(y, training=training)
        y = self.att(y, y,
                     use_causal_mask = True,
                     training=training)
        y = self.dropout1(y, training=training)
        x += y
        y = x
        y = self.layernorm2(y, training=training)
        y = self.ffn(y, training=training)
        y = self.dropout2(y, training=training)
        return x + y

In [5]:
#Positional encoding for self-attention

# Source pulled from KerasNLP: https://github.com/keras-team/keras-nlp/blob/v0.4.1/keras_nlp/layers/sine_position_encoding.py#L22
class SinePositionEncoding(keras.layers.Layer):
    """Sinusoidal positional encoding layer.
    This layer calculates the position encoding as a mix of sine and cosine
    functions with geometrically increasing wavelengths. Defined and formulized
    in [Attention is All You Need](https://arxiv.org/abs/1706.03762).
    Takes as input an embedded token tensor. The input must have shape
    [batch_size, sequence_length, feature_size]. This layer will return a
    positional encoding the same size as the embedded token tensor, which
    can be added directly to the embedded token tensor.
    This layer optionally accepts `tf.RaggedTensor`s as inputs to process
    batches of sequences of different lengths. The one ragged dimension must be
    the dimension that corresponds to the sequence, that is, the penultimate
    dimension.
    Args:
        max_wavelength: The maximum angular wavelength of the sine/cosine
            curves, as described in Attention is All You Need. Defaults to
            10000.
    Examples:
    ```python
    # create a simple embedding layer with sinusoidal positional encoding
    seq_len = 100
    vocab_size = 1000
    embedding_dim = 32
    inputs = keras.Input((seq_len,), dtype=tf.float32)
    embedding = keras.layers.Embedding(
        input_dim=vocab_size, output_dim=embedding_dim
    )(inputs)
    positional_encoding = keras_nlp.layers.SinePositionEncoding()(embedding)
    outputs = embedding + positional_encoding
    ```
    References:
     - [Vaswani et al., 2017](https://arxiv.org/abs/1706.03762)
    """

    def __init__(
        self,
        max_wavelength=10000,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.max_wavelength = max_wavelength

    def call(self, inputs):
        # TODO(jbischof): replace `hidden_size` with`hidden_dim` for consistency
        # with other layers.
        if isinstance(inputs, tf.RaggedTensor):
            bounding_shape = inputs.bounding_shape()
            position_embeddings = (
                self._compute_trim_and_broadcast_position_embeddings(
                    bounding_shape,
                )
            )
            # then apply row lengths to recreate the same ragged shape as inputs
            return tf.RaggedTensor.from_tensor(
                position_embeddings,
                inputs.nested_row_lengths(),
            )
        else:
            return self._compute_trim_and_broadcast_position_embeddings(
                tf.shape(inputs),
            )

    def _compute_trim_and_broadcast_position_embeddings(self, shape):
        seq_length = shape[-2]
        hidden_size = shape[-1]
        position = tf.cast(tf.range(seq_length), self.compute_dtype)
        min_freq = tf.cast(1 / self.max_wavelength, dtype=self.compute_dtype)
        timescales = tf.pow(
            min_freq,
            tf.cast(2 * (tf.range(hidden_size) // 2), self.compute_dtype)
            / tf.cast(hidden_size, self.compute_dtype),
        )
        angles = tf.expand_dims(position, 1) * tf.expand_dims(timescales, 0)
        # even indices are sine, odd are cosine
        cos_mask = tf.cast(tf.range(hidden_size) % 2, self.compute_dtype)
        sin_mask = 1 - cos_mask
        # embedding shape is [seq_length, hidden_size]
        positional_encodings = (
            tf.sin(angles) * sin_mask + tf.cos(angles) * cos_mask
        )

        return tf.broadcast_to(positional_encodings, shape)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "max_wavelength": self.max_wavelength,
            }
        )
        return config

In [6]:
#Token and position embedding to allow the model to differentiate token positions, for self-attention
class MaskedTokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, input_dim, output_dim, **kwargs):
        super(MaskedTokenAndPositionEmbedding, self).__init__(**kwargs)
        self.token_emb = keras.layers.Embedding(input_dim=input_dim,
                                                output_dim=output_dim,
                                                mask_zero=True)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen+1,
                                              output_dim=output_dim,
                                              mask_zero=True)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=1, limit=maxlen+1, delta=1)
        positions = positions * tf.cast(tf.sign(x),tf.int32)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [7]:
#Sine position embeddings, for self-attention
class MaskedTokenAndSinePositionEmbedding(keras.layers.Layer):
    def __init__(self, input_dim, output_dim, max_wavelength=10000,**kwargs):
        super(MaskedTokenAndSinePositionEmbedding, self).__init__(**kwargs)
        self.token_emb = keras.layers.Embedding(input_dim=input_dim,
                                                output_dim=output_dim,
                                                mask_zero=True)
        self.pos_emb = SinePositionEncoding(max_wavelength=max_wavelength)

    def call(self, x):
        mask = tf.expand_dims(tf.sign(x),-1)
        x = self.token_emb(x)
        positions = self.pos_emb(x)
        positions = positions * mask
        return x + positions

In [8]:
# Custom masked loss/accuracy functions
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def MaskedSparseCategoricalCrossentropy(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

def MaskedSparseCategoricalAccuracy(real, pred):
    accuracies = tf.equal(tf.cast(real,tf.int64), tf.argmax(pred, axis=2))
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    accuracies = tf.math.logical_and(mask, accuracies)
    accuracies = tf.cast(accuracies, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)


In [9]:
 !wget https://www.cs.mtsu.edu/~jphillips/courses/CSCI4850-5850/public/Austen.txt

--2023-06-19 07:12:46--  https://www.cs.mtsu.edu/~jphillips/courses/CSCI4850-5850/public/Austen.txt
Resolving www.cs.mtsu.edu (www.cs.mtsu.edu)... 161.45.162.100
Connecting to www.cs.mtsu.edu (www.cs.mtsu.edu)|161.45.162.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4340030 (4.1M) [text/plain]
Saving to: ‘Austen.txt’

Austen.txt          100%[===================>]   4.14M  --.-KB/s    in 0.04s   

2023-06-19 07:12:46 (102 MB/s) - ‘Austen.txt’ saved [4340030/4340030]



In [10]:
segment_size = 20
model_length = 10*segment_size
batch_size = 100

In [11]:
with open('Austen.txt', 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
text = []
j = 0
for i in range(len(lines)-1):
    if lines[i] == '':
        line = ' '.join(lines[j:i])
        if line != '':
            text = text + [line[k:k+model_length-2] for k in range(0, len(line), model_length-2)]
            # text = text + [line]
        j = i+1

In [12]:
len(text)

In [13]:
text[0]

'Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to unite some of the best blessings of existence; and had lived nearly twenty-one years in the world'

In [14]:
# Subset? - All of the data will take some time...
n_seq = len(text)
n_seq = 1000
split_point = int(n_seq * 0.8)
text = text[:n_seq]
np.random.shuffle(text) # In-place modification
max_length = np.max([len(i) for i in text])+2 # Start+stop
max_length

200

In [15]:
#Encode / Decode tokens using built-in functions
#Using Parker's filter
def remove_filler(val):
    if val!=(len(tokenizer)-1):
        return True
    else:
        return False

def encode_seq(x, tokenizer, max_length=0):
    #String to token IDs
    #Using "0" as padded token
    encoded = tokenizer.encode(x, max_length=max_length, truncation=True)
    padding = encoded + [len(tokenizer)-1 for i in range (max_length - len(encoded))]
    return padding

def decode_seq(x, tokenizer):
    #Token IDs to string
    filtered_text =filter(remove_filler,x)
    filler_removed = list(filtered_text)
    return tokenizer.decode(filler_removed)

In [16]:
temp = encode_seq(text[0], tokenizer, max_length)
print(*temp)

1 2061 3297 286 2045 582 318 1770 13 5780 1701 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256

In [17]:
len(temp)

In [18]:
decoded_temp = decode_seq(temp, tokenizer)
print(decoded_temp)

"What sort of looking man is Mr. Martin?"


In [19]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data, max_length, batch_size=32, **kwargs):
        super(DataGenerator, self).__init__(**kwargs)
        'Initialization'
        self.batch_size = batch_size
        self.data = data # a handle only...
        self.indices = np.arange(self.data[0].shape[0])
        self.max_length = max_length
        self.idx = 0
        self.on_epoch_end()

    def __len__(self):
        'The number of batches per epoch'
        return int(np.floor(self.data[0].shape[0] / self.batch_size))

    def __getitem__(self, index):
        'Generate one (enumerated) batch of data'
        # Generate indices for a batch and grab batch
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        return self.__data_generation(indices)

    def __data_generation(self, ids):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Generate data
        x = tf.convert_to_tensor(self.data[0][ids],dtype=tf.int32)
        y = tf.convert_to_tensor(self.data[1][ids],dtype=tf.int32)
        return x, y

    def on_epoch_end(self):
        return np.random.shuffle(self.indices)

    ## Needed for TF Dataset conversion...
    def output_signature(self):
        sig = self[0]
        return (tf.TensorSpec.from_tensor(sig[0]),
                tf.TensorSpec.from_tensor(sig[1]))

In [20]:
#Encode in batches
X = np.vstack([encode_seq(x,tokenizer,max_length + (segment_size - (max_length-1) % segment_size)) for x in text])
training = DataGenerator((X[:split_point,:-1],
                          X[:split_point,1:]),model_length,batch_size)
validation = DataGenerator((X[split_point:,:-1],
                            X[split_point:,1:]),model_length,batch_size)

In [21]:
len(training)

In [22]:
temp_length = len(temp)
temp = np.array([temp_length+1 for i in temp])
temp

array([201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201,
       201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201,
       201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201,
       201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201,
       201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201,
       201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201,
       201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201,
       201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201,
       201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201,
       201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201,
       201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201,
       201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201,
       201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201,
       201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 201, 20

In [23]:
np.array([np.random.randint(i) for i in temp])-100

array([  64,  -17,   81,  -46,  -38,   36,   49, -100,  -66,   79,   86,
        -84,   80,  -99,  -38,  -51,  -33, -100,   63,  -96,   93,   91,
         98,  -92,   91,  -87,   -8,  -71,  -63,   51,   84,  -53,  -80,
          4,   55,   41,   20,   68,  -55,   37,   80,  100,  -19,    7,
         72,  -87,  -73,   28,    4,  -31,  -26,   40,   75,   96,  -15,
        -78,    3,  -65,   45,  -70,   81,    0,  -41,  -19,   52,   87,
        -18,  -31,  -70,    9,  -30,  -22,   64,  -82,  -21,   53,  -17,
        -79,  -66,  -82,   64,    7,   18,    1,  -62,   55,   87,  -28,
        -18,   23,   88,  -71,   32,  -34,   52,   19,   31,   82,   34,
         80,   34,    2,   13,   45,    7,   73,   -8,   56,   54,   91,
         -3,  -83,  -56,   36,   72,  -87,   77,  -63,   54,    9,  -31,
         14,    3,   56,  -81,   57,   80,  -56,   73,   19,   68,   33,
         79,   -3,  -53,   49,  -87,  -30,  -10,   23,  -23,   36,    0,
        -58,  -54,   94,   31,   38,   55,   -5,   

In [24]:
my_batch = training[0]
my_batch[0].shape

2023-06-19 07:12:59.021991: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-06-19 07:12:59.022038: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-06-19 07:12:59.022073: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-cwk2p): /proc/driver/nvidia/version does not exist
2023-06-19 07:12:59.022587: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorShape([100, 200])

In [25]:
#Finalize and compile model
n_tokens = len(tokenizer)
embedding_size = 128
stack = 5
num_heads = 12
memory_size = segment_size * 3

y = x = keras.layers.Input((None,))
y = MaskedTokenAndSinePositionEmbedding(input_dim=n_tokens,
                                        output_dim=embedding_size)(y)
for _ in range(stack):
    y = GPTransformerBlock(embedding_size,
                           num_heads,
                           embedding_size*2)(y)

y = keras.layers.Dense(n_tokens)(y)

model = keras.Model(x,y)
#model.compile(loss=MaskedSparseCategoricalCrossentropy,
#              optimizer=keras.optimizers.Adam(),
#              metrics=MaskedSparseCategoricalAccuracy)
#model.summary()
#keras.utils.plot_model(model,show_shapes=True,expand_nested=True)

In [26]:
#started at 8:00pm 6/15
#server disconnect at ~5:00am
#reconnect at 7:00am 
#finished at 9:10am 6/16

In [27]:
!curl -O https://s-stem-data.nyc3.digitaloceanspaces.com/Keiningham-Weights/Keiningham-Weights-2.tar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  196M  100  196M    0     0  78.4M      0  0:00:02  0:00:02 --:--:-- 78.4M


In [28]:
!tar -xf Keiningham-Weights-2.tar

In [29]:
model.load_weights("Keiningham-Weights-2")

In [30]:
# What should we see?
i = 0
print('Input:', training[0][0][i])
print('Output:', training[0][1][i])

Input: tf.Tensor(
[    1  5297    11 20461    64    11   340   318  3194   503   287   674
  1218  2443    13   775 18984   340   422   262   412  1455   415 29677
    82    13   632   373 46619   624   338    11   345   760   526 50256
 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256
 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256
 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256
 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256
 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256
 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256
 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256
 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256
 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256
 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256 50256
 50256 50256 50256 50256 50256 50

In [31]:
# Teacher-forcing
result = model(training[0][0][i:i+1]).numpy()
result.argmax(-1)[0]

array([   40,    11, 20461,    64,    11,   340,   318,  3194,   503,
         287,   674,  1218,  2443,    13,   775, 18984,   340,   422,
         262,   412,  1455,   415, 29677,    82,    13,   632,   373,
       46619,   624,   338,    11,   345,   760,   526, 50256, 50256,
       50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
       50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
       50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
       50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
       50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
       50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
       50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
       50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
       50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
       50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
       50256, 50256,

In [32]:
decode_seq(result.argmax(-1)[0],tokenizer)

'I, papa, it is written out in our second page. We copied it from the Elegant Extracts. It was Garrick\'s, you know."'

In [33]:
# How much agreement really? - remember masked tokens (0) don't count
MaskedSparseCategoricalAccuracy(training[0][1][i:i+1],result).numpy()*100.0

In [34]:
def select_token(x):
    x = np.cumsum(x)
    return np.argmax(x > np.random.random())

In [39]:
# Turn off teacher forcing...
# Prompt (needs to be at least 1 - the start token?...)
i = 1
data = training[0][0][i:i+1]
prompt = 6
tokens = np.full(data.shape,dtype=np.int32,fill_value=50256) #Courtesy of Parker Daniels 
tokens[0,0:prompt] = data[:,0:prompt]

print("Original:")
print(decode_seq(data[0],tokenizer))
print()

print("Prompt:")
print(decode_seq(tokens[0],tokenizer))
print()

print("Decoding:")
print(decode_seq(tokens[0],tokenizer),end='')
for x in range(prompt,data.shape[1]-1):
    probabilities = keras.activations.softmax(model(tokens)).numpy()[0,x-1]
    # Most likely token...
    result = probabilities.argmax(-1)
    # Sampled token...
    result = np.apply_along_axis(select_token, -1, probabilities)
    tokens[0,x] = result
    if result == len(tokenizer)-1:
        break # Stop token found!
    print(decode_seq(tokens[0,x:x+1],tokenizer),end='')
print()
print()

result = model(tokens).numpy()
print("Remodeled:")
print(decode_seq(result.argmax(-1)[0],tokenizer))

Original:
"So do I," said Mrs. Weston gently, "very much."

Prompt:
"So do I," said

Decoding:
"So do I," said Mrs. Weston gently, "very much."

Remodeled:
I do I," said Mrs. Weston gently, "very much."


In [40]:
# Turn off teacher forcing...
# Prompt (needs to be at least 1 - the start token?...)
i = 3
data = training[0][0][i:i+1]
prompt = 4
tokens = np.full(data.shape,dtype=np.int32,fill_value=50256) #Courtesy of Parker Daniels
tokens[0,0:prompt] = data[:,0:prompt]

print("Original:")
print(decode_seq(data[0],tokenizer))
print()

print("Prompt:")
print(decode_seq(tokens[0],tokenizer))
print()

print("Decoding:")
print(decode_seq(tokens[0],tokenizer),end='')
for x in range(prompt,data.shape[1]-1):
    probabilities = keras.activations.softmax(model(tokens)).numpy()[0,x-1]
    # Most likely token...
    result = probabilities.argmax(-1)
    # Sampled token...
    result = np.apply_along_axis(select_token, -1, probabilities)
    tokens[0,x] = result
    if result == len(tokenizer)-1:
        break # Stop token found!
    print(decode_seq(tokens[0,x:x+1],tokenizer),end='')
print()
print()

result = model(tokens).numpy()
print("Remodeled:")
print(decode_seq(result.argmax(-1)[0],tokenizer))

Original:
n.

Prompt:
n.

Decoding:
n.

Remodeled:
f He


In [37]:
# Turn off teacher forcing...
# Prompt (needs to be at least 1 - the start token?...)
sample = "It is a truth universally acknowledged"
data = np.array(encode_seq(sample, tokenizer, max_length))
data = data.reshape(1,-1)
prompt = len(sample.split())
tokens = np.full(data.shape,dtype=np.int32,fill_value=len(tokenizer)-1) #Courtesy of Parker Daniels
tokens[0,0:prompt] = data[:,0:prompt]

print("Original:")
print(decode_seq(data[0],tokenizer))
print()

print("Prompt:")
print(decode_seq(tokens[0],tokenizer))
print()

print("Decoding:")
print(decode_seq(tokens[0],tokenizer),end='')
for x in range(prompt,data.shape[1]-1):
    probabilities = keras.activations.softmax(model(tokens)).numpy()[0,x-1]
    # Most likely token...
    result = probabilities.argmax(-1)
    # Sampled token...
    result = np.apply_along_axis(select_token, -1, probabilities)
    tokens[0,x] = result
    if result == len(tokenizer)-1:
        break # Stop token found!
    print(decode_seq(tokens[0,x:x+1],tokenizer),end='')
print()
print()

result = model(tokens).numpy()
print("Remodeled:")
print(decode_seq(result.argmax(-1)[0],tokenizer))

Original:
It is a truth univ

Prompt:
It is a truth un

Decoding:
It is a truth un just kind about hear to be able with take all the respectable; with me at a better than than to have said. She can be in your name, often himself to suppose herself, as a man,

Remodeled:
 was only very Emma just kind for hear to be able to take all the respectable; for me at present narrow than than have could said at I can be in any name's often himself, be you, as a man, s


In [38]:
#It seems to have memorized the original text
#The model does suffer for short prompts
#Custom prompts are interesting